In [1]:
import torch
import pandas as pd
from datetime import timedelta
from datetime import datetime

import numpy as np

from nera.data import *
from nera.models.ratings import EloAnalytical, EloManual, EloNumerical
from nera.reference import *
from nera.trainer import Trainer
from nera.utils import print_rating_diff


/home/kubos/anaconda3/envs/bp/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
da = DataAcquisition()
df = da.get_data(FROM_CSV, fname="../resources/other_leagues.csv")
df['DT'] = pd.to_datetime(df['DT'], format="%Y-%m-%d %H:%M:%S")
data_transform = DataTransformation(df, timedelta(365))
df = df[(df['League'] != 'EuroLeague') & (df['League'] != 'EuroCup')] 
df = df.reset_index()

transform = DataTransformation(df, timedelta(365))
dataset = transform.get_dataset(node_f_extract=False, edge_f_one_hot=True)

team_count = transform.num_teams

2024-03-01 15:09:14.035 | INFO     | nera.data._data_saving_loading:load_data_csv:70 - 21100 rows loaded from ../resources/other_leagues.csv


### Manual Elo with NN pass

In [3]:
# dummy dataset

delta = timedelta(days=365)
delta2 = timedelta(days=366)
now = datetime.now()
data = pd.DataFrame({'DT': [*(3 * [now]), *(3 * [now - delta2]), *(3 * [now - 2 * delta2]), *(3 * [now - 3 * delta2])], 
                     'Home': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
                     'Away': ['B', 'C', 'A', 'C', 'A', 'B', 'B', 'C', 'A', 'C', 'A', 'B'],
                     'Winner': ['home', 'away', 'away', 'home', 'away', 'away', 'home', 'home', 'away', 'home', 'away', 'home'],
                     'Home_points': [10, 5, 15, 12, 15, 6, 20, 10, 10, 14, 3, 12],
                     'Away_points': [4, 8, 17, 10, 16, 14, 18, 9, 15, 0, 11, 4],
                     'League': [*(12 * ['liga'])],
                     })

transform2 = DataTransformation(data, timedelta(days=365))
dummy_dataset = transform2.get_dataset(edge_f_one_hot=True)

In [4]:
elo_man = EloManual(team_count=transform2.num_teams)
trainer = Trainer(dummy_dataset, elo_man, train_ratio=1)
reference_maker = RatingReference(data, transform2.team_mapping)

print('Computed elo Dummy: ', reference_maker.compute_reference('elo')[0])
acc_dummy = trainer.train(epochs=1)
print(elo_man.elo)
print()
print('----------------------------------------')
print()

reference_maker.matches = (df, transform.team_mapping)
computed = reference_maker.compute_reference('elo')[0]

print()
elo = EloManual(team_count=transform.num_teams)
trainer.dataset = dataset
trainer.model = elo
acc_late = trainer.train(epochs=1)

err = False
for i in range(len(computed)):
    cmp = float(computed[transform.team_mapping[transform.inv_team_mapping[i]]])
    net = float(elo.elo[transform.team_mapping[transform.inv_team_mapping[i]]])
    if cmp - net > 0.1:
        print(rf'ERROR on index {i}:: {cmp} / {net}')
        err = True
    if i < 5:
        print(f'{i}:: computed: {cmp:10.3f} || net: {net:10.3f}')
if not err:
    print("...")
    print("[SUCCESS]: Computed elo is the same as Elo from NN")



Computed elo Dummy:  [1450.41466185  783.32864878  766.25668937]
Parameter containing:
tensor([1450.4147,  783.3286,  766.2567], dtype=torch.float64,
       requires_grad=True)

----------------------------------------
0:: computed:   1379.688 || net:   1379.688
1:: computed:   2976.967 || net:   2976.967
2:: computed:    384.481 || net:    384.481
3:: computed:    571.470 || net:    571.470
4:: computed:   1309.883 || net:   1309.883
...
[SUCCESS]: Computed elo is the same as Elo from NN


### Elo with gradient

first lets check our analytical backward pass

In [6]:
from torch.autograd import gradcheck
from nera.models.ratings._elo._analytical import elo_function
# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True))
test = gradcheck(elo_function, input, eps=1e-6, atol=1e-4)
print(test)

True


Define training function for elo

Now, let's train elo with analytical backward pass

In [7]:
elo_grad = EloAnalytical(team_count, cd_grad=False, c=torch.tensor(3, dtype=torch.float64), d=torch.tensor(500., dtype=torch.float64))
trainer.model = elo_grad
trainer.train(verbose=True)

2024-03-01 15:09:45.848 | INFO     | nera.trainer:_train_rating:166 - [TRN]  Epoch: 0, training loss: 1762432.146, training accuracy: 47.81% 
ratings (first 5): tensor([ 2378.9668, 15953.9127, -7156.4009, -9066.7111,  3766.2664],
       dtype=torch.float64, grad_fn=<SliceBackward0>)


array([47.81268067])

And Elo with autograd (numerical gradient backward pass)

In [8]:
elo_auto = EloNumerical(team_count, cd_grad=False, c=torch.tensor(3, dtype=torch.float64), d=torch.tensor(500., dtype=torch.float64))
trainer.model = elo_auto
trainer.train(verbose=True)

2024-03-01 15:09:53.915 | INFO     | nera.trainer:_train_rating:166 - [TRN]  Epoch: 0, training loss: 1762432.061, training accuracy: 47.81% 
ratings (first 5): tensor([ 2378.9668, 15953.9135, -7156.4447, -9066.7181,  3766.0936],
       dtype=torch.float64, grad_fn=<SliceBackward0>)


array([47.81268067])

And finally, compare wheter these two are the same
___________________

In [9]:
print_rating_diff(elo_auto, elo_grad, transform)

Ratings diff:
Rating 0:
0:: computed:   2378.967 || net:   2378.967
1:: computed:  15953.913 || net:  15953.913
[ERROR] in rating 0 on index 2:: -7156.444672822601 / -7156.400875638662
[ERROR] in rating 0 on index 3:: -9066.718140522222 / -9066.711086527102
[ERROR] in rating 0 on index 4:: 3766.093552898145 / 3766.2664314509893
...

Total number of errors: 53 out of 146 computed ratings
Cumulative sum of errors: 58.48463412058629
Average difference: 1.1034836626525715
Average difference percentage: 0.036955311810839636%
Maximal difference: 24.77435875706351
-------------------------

Hyperparams diff:


Now let's see what happens, when we let the model learn the c, d metaparameters as well

In [10]:
elo_auto2 = EloNumerical(team_count, hp_grad=True, c=torch.tensor(5000, dtype=torch.float64), d=torch.tensor(50000, dtype=torch.float64))
trainer.model = elo_auto2
trainer.train(verbose=True)

2024-03-01 15:10:04.935 | INFO     | nera.trainer:_train_rating:166 - [TRN]  Epoch: 0, training loss: 1026752.963, training accuracy: 46.98% 
ratings (first 5): tensor([ 2280.6030,  3506.5187, -1489.0913, -1359.1684,   283.4601],
       dtype=torch.float64, grad_fn=<SliceBackward0>)


array([46.97758078])

In [11]:
elo_grad2 = EloAnalytical(team_count, hp_grad=True, c=torch.tensor(5000., dtype=torch.float64), d=torch.tensor(50000, dtype=torch.float64))
trainer.model = elo_grad2
trainer.train(verbose=True)

2024-03-01 15:10:15.289 | INFO     | nera.trainer:_train_rating:166 - [TRN]  Epoch: 0, training loss: 1026752.963, training accuracy: 46.98% 
ratings (first 5): tensor([ 2280.6030,  3506.5187, -1489.0913, -1359.1684,   283.4601],
       dtype=torch.float64, grad_fn=<SliceBackward0>)


array([46.97758078])

In [12]:
print_rating_diff(elo_auto2, elo_grad2, transform)

Ratings diff:
Rating 0:
0:: computed:   2280.603 || net:   2280.603
1:: computed:   3506.519 || net:   3506.519
2:: computed:  -1489.091 || net:  -1489.091
3:: computed:  -1359.168 || net:  -1359.168
4:: computed:    283.460 || net:    283.460
...

[SUCCESS]: All  ratings computed analytically and numerically are the SAME
-------------------------

Hyperparams diff:
Numerical: hyperparam[0] = 4984.004 :: Numerical: hyperparam[0] = 4984.004
Numerical: hyperparam[1] = 50015.993 :: Numerical: hyperparam[1] = 50015.993
